In [36]:
# importing all necessary libraries


from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os, numpy as np
from PIL import Image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense


from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
# Parameters
IMG_SIZE = (64, 64)
CLASSES = ['forest', 'mountain', 'street']

In [14]:
# Load images
def load_data(data_dir, classes, img_size):
    images, labels = [], []
    for label, cls in enumerate(classes):
        path = os.path.join(data_dir, cls)
        for file in os.listdir(path):
            img = Image.open(os.path.join(path, file)).resize(img_size)
            images.append(np.array(img))
            labels.append(label)
    return np.array(images), np.array(labels)

X, y = load_data('seg_train', CLASSES, IMG_SIZE)

In [16]:
# Normalize pixel values
X = X / 255.0

In [18]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)

In [24]:
# model building using CNN
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(3, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 18s 89ms/step - accuracy: 0.7092 - loss: 0.6429 - val_accuracy: 0.8828 - val_loss: 0.3142
Epoch 2/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 17s 92ms/step - accuracy: 0.9105 - loss: 0.2492 - val_accuracy: 0.9051 - val_loss: 0.2600
Epoch 3/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 16s 89ms/step - accuracy: 0.9244 - loss: 0.2086 - val_accuracy: 0.9163 - val_loss: 0.2373
Epoch 4/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 19s 103ms/step - accuracy: 0.9409 - loss: 0.1653 - val_accuracy: 0.9253 - val_loss: 0.2264
Epoch 5/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 18s 100ms/step - accuracy: 0.9544 - loss: 0.1336 - val_accuracy: 0.9309 - val_loss: 0.2031
Epoch 6/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 18s 102ms/step - accuracy: 0.9746 - loss: 0.0882 - val_accuracy: 0.9316 - val_loss: 0.2153
Epoch 7/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 22s 120ms/step - accuracy: 0.9804 - loss: 0.0646 - val_accuracy: 0.9288 - val_loss: 0.2426
Epoch 8/10
180/180 ━━━━━━━━━━━━━━━━━━━━ 22s 120ms/step - accuracy: 0.9847 - loss: 0.05

In [28]:
 # Evaluate on test set
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

45/45 ━━━━━━━━━━━━━━━━━━━━ 2s 33ms/step - accuracy: 0.8988 - loss: 0.4247
Test Accuracy: 90.51%


In [30]:
# Classification report
from sklearn.metrics import classification_report
y_pred = model.predict(X_test).argmax(axis=1)
print(classification_report(y_test, y_pred, target_names=CLASSES))

45/45 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step
              precision    recall  f1-score   support

      forest       0.94      0.92      0.93       454
    mountain       0.85      1.00      0.92       503
      street       0.94      0.79      0.86       476

    accuracy                           0.91      1433
   macro avg       0.91      0.90      0.90      1433
weighted avg       0.91      0.91      0.90      1433



In [38]:
# Save metrics
with open("metrics.json", "w") as f:
    json.dump({"test_accuracy": test_accuracy}, f)
    

In [40]:
# Save model
model.save("model.h5")